In [1]:
from autumn.infrastructure.remote import springboard
from autumn.projects.sm_covid2.common_school.runner_tools import (
    run_full_analysis, 
    print_continuous_status, 
    download_analysis,
    DEFAULT_RUN_CONFIG,
)

c:\Users\rrag0004\.conda\envs\summer2\lib\site-packages\summer\runner\vectorized_runner.py:363: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def get_strain_infection_values(
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### Define task function

#### Standard config

In [2]:
DEFAULT_RUN_CONFIG

{'N_CHAINS': 8,
 'N_OPTI_SEARCHES': 16,
 'OPTI_BUDGET': 10000,
 'METROPOLIS_TUNE': 5000,
 'METROPOLIS_DRAWS': 30000,
 'METROPOLIS_METHOD': 'DEMetropolisZ',
 'FULL_RUNS_SAMPLES': 1000,
 'BURN_IN': 20000}

In [3]:
ISO3 = "FRA"
ANALYSIS = "main"

N_CORES = DEFAULT_RUN_CONFIG['N_CORES']

N_CHAINS = DEFAULT_RUN_CONFIG['N_CHAINS']
N_OPTI_SEARCHES = DEFAULT_RUN_CONFIG['N_OPTI_SEARCHES']
OPTI_BUDGET = DEFAULT_RUN_CONFIG['OPTI_BUDGET']

METROPOLIS_TUNE = DEFAULT_RUN_CONFIG['METROPOLIS_TUNE']
METROPOLIS_DRAWS = DEFAULT_RUN_CONFIG['METROPOLIS_DRAWS']
METROPOLIS_METHOD = DEFAULT_RUN_CONFIG['METROPOLIS_METHOD']

FULL_RUNS_SAMPLES = DEFAULT_RUN_CONFIG['FULL_RUNS_SAMPLES']
BURN_IN = DEFAULT_RUN_CONFIG['BURN_IN']

### Check that configuration is relevant

In [4]:
assert (METROPOLIS_DRAWS - BURN_IN) * N_CHAINS >= FULL_RUNS_SAMPLES
assert METROPOLIS_METHOD in ("DEMetropolis", "DEMetropolisZ")

In [5]:
def remote_full_analysis_task(bridge: springboard.task.TaskBridge, iso3: str = 'FRA', analysis: str = "main"):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')

    bridge.logger.info(f"Running full analysis for {iso3}. Analysis type: {analysis}.")

    idata, uncertainty_df, diff_quantiles_df = run_full_analysis(
        iso3,
        analysis=analysis, 
        n_cores=N_CORES,
        opti_params={'n_searches': N_OPTI_SEARCHES, 'search_iterations': OPTI_BUDGET},
        mcmc_params={'draws': METROPOLIS_DRAWS, 'tune': METROPOLIS_TUNE, 'chains': N_CHAINS, 'method': METROPOLIS_METHOD},
        full_run_params={'samples': FULL_RUNS_SAMPLES, 'burn_in': BURN_IN},
        output_folder=bridge.out_path,
        logger=bridge.logger
    )
   
    bridge.logger.info("Full analysis complete")

In [6]:
mspec = springboard.EC2MachineSpec(N_CORES, 4, "compute")
task_kwargs = {
    "iso3": ISO3,
    "analysis": ANALYSIS
}
tspec = springboard.TaskSpec(remote_full_analysis_task, task_kwargs)

In [7]:
analysis_title = "test_4cpu"
config_str = f"_{ANALYSIS}_LHS{N_OPTI_SEARCHES}_opt{OPTI_BUDGET}_mc{METROPOLIS_TUNE}n{METROPOLIS_DRAWS}"

run_path = springboard.launch.get_autumn_project_run_path("school_project", ISO3, analysis_title + config_str)
run_path

'projects/school_project/FRA/2023-09-11T1117-test_4cpu_main_LHS16_opt10000_mc5000n30000'

In [8]:
runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path)

SSH connection still waiting, retrying 1/5


In [15]:
runner.s3.get_status()

'SUCCESS'

In [ ]:
print(runner.top("%MEM"))

In [ ]:
# wait function with status printing
# print_continuous_status(runner)

In [ ]:
# run_path = 'projects/school_project/FRA/2023-08-22T1004-single_start_main_LHS4_opt10000_mc2000n10000'

In [16]:
download_analysis(run_path)